# Load all the pig dicom data and convert them to nifti

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
# !pip install loguru pydicom scipy scikit-image

# !pip install git+https://github.com/mjirik/io3d --upgrade --force-reinstall
# !pip install git+https://github.com/mjirik/imma --upgrade --force-reinstall

# !pip install git+https://github.com/mjirik/io3d
# !pip install git+https://github.com/mjirik/imma



In [4]:
import io3d
from pathlib import Path
from loguru import logger
from pprint import pprint, pformat
import pandas as pd
import re
import numpy as np
import json
import tqdm

logger.enable("io3d")
force = True
# force = False

recreate_meta = False
recreate_meta = True
# base_path = Path(r"H:\biomedical\orig\pilsen_pigs_all\transplantation_dicom")
base_path = Path(r"~/mnt/nas-bmc3_ct/Prasata Tx").expanduser()
# base_path = Path(r"~/Downloads/_temp/").expanduser()  #: used when the zip files are downloaded manually
# raw_path = Path(r"H:\biomedical\orig\pilsen_pigs_all\transplantation_nii_transposed")
# transposed_path = Path(r"H:\biomedical\orig\pilsen_pigs_all\transplantation_nii_transposed")
# output_path = Path(r"H:\biomedical\orig\pilsen_pigs")
output_path= Path(r"~/mnt/nas-bmc3_ct/pilsen_pigs_all/transplantation_nii_transposed").expanduser()

assert base_path.exists()

# Convert all to nii

In [5]:

def touch_file(pth:Path):
    try:
        with open(pth, "rb") as f:
            # read just some part of the file
            f.read(1)
    except Exception as e:
        import traceback
        print(f"Error in touching file {pth}: {e}")
        traceback.print_exc()

In [6]:
from typing import Union

def get_projection(
        datap:io3d.image.DataPlus, axis:Union[int,str], method:str="max"
):
    """Get projection of 3D data to 2D."""
    if isinstance(axis, str):
        dict_axis = {"axial": 0, "coronal": 1, "sagittal": 2}
        if axis in dict_axis:
            axis = dict_axis[axis]
        else:
            raise ValueError(f"Unknown axis {axis}, use one of {list(dict_axis.keys())} or 0, 1, 2")

    data3d = datap.data3d
    axcodes = datap.orientation_axcodes
    data3d = io3d.image.transform_orientation(data3d, axcodes, "SPL")
    if method == "max":
        data2d = data3d.max(axis=axis)
    elif method == "mean":
        data2d = data3d.mean(axis=axis)
    else:
        raise ValueError(f"Unknown method {method}")
    return data2d



In [7]:
# data2d = get_projection(datap, 0, "max")
# from matplotlib import pyplot as plt
# plt.style.use('classic')
# plt.imshow(data2d, cmap="gray_r")
# plt.colorbar()
# plt.style.available


In [8]:
import tqdm

In [9]:
# List all directories under base_path (recursively)
leaf_dirs = []
for d in tqdm.tqdm(list(base_path.glob("**/"))):
    if d.is_dir():
        # Get direct children
        children = list(d.iterdir())
        # Identify if there are any subdirectories
        has_subdir = any(child.is_dir() for child in children)
        # Identify if there is at least one file in the directory
        has_file = any(child.is_file() for child in children)
        # A leaf directory has no subdirectories but does contain files
        if not has_subdir and has_file:
            leaf_dirs.append(d)

leaf_dirs

100%|██████████| 1791/1791 [04:04<00:00,  7.32it/s]


[PosixPath('/home/mjirik/mnt/nas-bmc3_ct/Prasata Tx/10_2019/Tx016A 14.10.2019/DICOM/20120814/26270000'),
 PosixPath('/home/mjirik/mnt/nas-bmc3_ct/Prasata Tx/10_2019/Tx016A 14.10.2019/DICOM/20120814/26270001'),
 PosixPath('/home/mjirik/mnt/nas-bmc3_ct/Prasata Tx/10_2019/Tx016A 14.10.2019/DICOM/20120814/26270002'),
 PosixPath('/home/mjirik/mnt/nas-bmc3_ct/Prasata Tx/10_2019/Tx016A 14.10.2019/DICOM/20120814/26270003'),
 PosixPath('/home/mjirik/mnt/nas-bmc3_ct/Prasata Tx/10_2019/Tx016A 14.10.2019/DICOM/20120814/26270004'),
 PosixPath('/home/mjirik/mnt/nas-bmc3_ct/Prasata Tx/10_2019/Tx016A 14.10.2019/DICOM/20120814/26270005'),
 PosixPath('/home/mjirik/mnt/nas-bmc3_ct/Prasata Tx/10_2019/Tx016A 14.10.2019/DICOM/20120814/26270006'),
 PosixPath('/home/mjirik/mnt/nas-bmc3_ct/Prasata Tx/10_2019/Tx016A 7.10.2019/DICOM/20120814/36330000'),
 PosixPath('/home/mjirik/mnt/nas-bmc3_ct/Prasata Tx/10_2019/Tx016A 7.10.2019/DICOM/20120814/36330001'),
 PosixPath('/home/mjirik/mnt/nas-bmc3_ct/Prasata Tx/10_20

In [14]:
fn = str(leaf_dirs[0].relative_to(base_path))

In [16]:
io3d.dcmreaddata.dicomdir_info(fn)

2025-04-30 13:25:10.294 | INFO     | io3d.dcmreaddata:files_in_dir:416 - completedirpath = /home/mjirik/mnt/nas-bmc3_ct/Prasata Tx/10_2019/Tx016A 14.10.2019/DICOM/20120814/26270000


AttributeError: module 'pydicom' has no attribute 'read_file'

In [15]:
# Get list of files

import os.path as op

fn = sorted(leaf_dirs)[0]
dd = io3d.dcmreaddata.DicomDirectory(str(fn))
dd.get_sorted_series_files(
    sort_keys=["SeriesNumber", "SliceLocation"],
    return_files_with_info=True,
    return_files=False,
)
dd

2025-04-30 13:22:40.101 | INFO     | io3d.dcmreaddata:files_in_dir:416 - completedirpath = /home/mjirik/mnt/nas-bmc3_ct/Prasata Tx/10_2019/Tx016A 14.10.2019/DICOM/20120814/26270000


AttributeError: module 'pydicom' has no attribute 'read_file'

In [12]:
io3d.read(sorted(leaf_dirs)[0], orientation_axcodes="IPL")

2025-04-30 12:50:45.167 | DEBUG    | io3d.datareader:__ReadFromDirectory:261 - Getting list of readable files in dir...
/home/mjirik/projects/io3d/.venv/lib/python3.12/site-packages/SimpleITK/extra.py:466: SyntaxWarning: invalid escape sequence '\s'
  """Blurs an image by separable convolution with discrete
ImageSeriesReader (0x385b35f0): Non uniform sampling or missing slices detected,  maximum nonuniformity:32.9579



{'series_number': 0,
 'datadir': '/home/mjirik/mnt/nas-bmc3_ct/Prasata Tx/10_2019/Tx016A 14.10.2019/DICOM/20120814/26270000',
 'voxelsize_mm': [0.04208416833667335, 0.5859375, 0.5859375],
 'orientation_axcodes': 'IPL',
 'data3d': array([[[-1000, -1001, -1000, ..., -1004, -1010, -1008],
         [ -999, -1007, -1005, ..., -1006, -1007, -1005],
         [ -995, -1008, -1010, ..., -1010, -1000,  -995],
         ...,
         [ -997,  -999,  -998, ...,  -999, -1000, -1007],
         [-1002,  -996,  -994, ..., -1005, -1007, -1007],
         [ -996,  -995,  -997, ..., -1002, -1007, -1003]],
 
        [[ -997, -1003, -1006, ...,  -999, -1000, -1005],
         [ -996, -1000, -1002, ...,  -999, -1001, -1003],
         [ -996,  -998, -1003, ..., -1003, -1001,  -997],
         ...,
         [ -999,  -998,  -996, ..., -1010, -1010, -1007],
         [-1009, -1002,  -996, ..., -1011, -1013, -1009],
         [-1008, -1003,  -998, ..., -1005, -1008, -1004]],
 
        [[-1000,  -998,  -998, ..., -1007

In [13]:
# fnlist = list(base_path.glob("*Tx0*D_V*"))
# find a leaf directory in base_path which contains some file



# fnlist = sorted(list(base_path.glob("*Tx0*D_A*")) + list(base_path.glob("*Tx0*D_V*")))[::-1]
fnlist = sorted(leaf_dirs)


print(f"Number of files: {len(fnlist)}")
from joblib import Parallel, delayed
import tqdm
#
for fn in tqdm.tqdm(fnlist):

    # take all files in parallel in the fn directory and open them and close, to intiate cloud syncronization
    # import Parallel, delayed
    # i want to have there the parent directory
    # fn_name = str(fn.relative_to(base_path)).replace("\\", "-").replace("/", "-")
    fn_name = str(fn.relative_to(base_path.parent)).replace("\\", "-").replace("/", "-")

    # logger.info(fn)
    fn_in = fn
    # fn_out = raw_path / fn.name / f"{fn.name}.mhd"
    fn_out = output_path / fn_name / f"{fn_name}.nii.gz"

    fn_out.parent.mkdir(parents=True, exist_ok=True)
    fn_meta = fn_out.parent / "meta.json"
    if force or (not fn_out.exists()):

        try:
            fn_fns = sorted(list(fn.glob("*")))
            tqdm.tqdm.write(f"Reading {len(fn_fns)} files in {fn_in} and writing {fn_out}")
            # tqdm.tqdm.write(f"Number of files in the directory: {len(fn_fns)}")
            # Parallel(n_jobs=2)(delayed(touch_file)(fn) for fn in tqdm.tqdm(fn_fns, desc="touching files"))

            axcodes = "IPL"
            # logger.debug(f"Reading {fn_in} with axcodes={axcodes}")
            datap = io3d.read(fn_in, series_number="first", orientation_axcodes=axcodes)
            # logger.debug(datap.keys())
            io3d.write(datap, fn_out)
            # logger.debug("writing done, creating projections")
            for axis in ["axial", "coronal", "sagittal"]:
                data2d = get_projection(datap, axis, "max")
                import skimage.io
                # change intensity to range 0..1

                data2d = (255 * (data2d - np.min(data2d)).astype(float) / (np.max(data2d) - np.min(data2d))).astype(np.uint8)
                skimage.io.imsave(fn_out.parent / f"{fn_out.stem}_{axis}.jpg", data2d)
            # logger.debug("projections done")
        except Exception as e:
            import traceback
            logger.error(f"Error in reading {fn_in}: {e}")
            traceback.print_exc()
            # logger.debug(f"shape={datap.data3d.shape}, {datap.orientation_axcodes}")
        # with open(fn_meta, "w") as f:
        #     json.dump(dict(row), f)





Number of files: 1405


  0%|          | 0/1405 [00:00<?, ?it/s]2025-04-30 12:51:34.328 | DEBUG    | io3d.datareader:__ReadFromDirectory:261 - Getting list of readable files in dir...


Reading 500 files in /home/mjirik/mnt/nas-bmc3_ct/Prasata Tx/10_2019/Tx016A 14.10.2019/DICOM/20120814/26270000 and writing /home/mjirik/mnt/nas-bmc3_ct/pilsen_pigs_all/transplantation_nii_transposed/Prasata Tx-10_2019-Tx016A 14.10.2019-DICOM-20120814-26270000/Prasata Tx-10_2019-Tx016A 14.10.2019-DICOM-20120814-26270000.nii.gz


ImageSeriesReader (0x385b35f0): Non uniform sampling or missing slices detected,  maximum nonuniformity:32.9579

  0%|          | 1/1405 [01:09<27:06:32, 69.51s/it]2025-04-30 12:52:44.119 | DEBUG    | io3d.datareader:__ReadFromDirectory:261 - Getting list of readable files in dir...


Reading 500 files in /home/mjirik/mnt/nas-bmc3_ct/Prasata Tx/10_2019/Tx016A 14.10.2019/DICOM/20120814/26270001 and writing /home/mjirik/mnt/nas-bmc3_ct/pilsen_pigs_all/transplantation_nii_transposed/Prasata Tx-10_2019-Tx016A 14.10.2019-DICOM-20120814-26270001/Prasata Tx-10_2019-Tx016A 14.10.2019-DICOM-20120814-26270001.nii.gz


...

Reading 500 files in /home/mjirik/mnt/nas-bmc3_ct/Prasata Tx/10_2019/Tx017A 4.10.2019/DICOM/20120814/29270009 and writing /home/mjirik/mnt/nas-bmc3_ct/pilsen_pigs_all/transplantation_nii_transposed/Prasata Tx-10_2019-Tx017A 4.10.2019-DICOM-20120814-29270009/Prasata Tx-10_2019-Tx017A 4.10.2019-DICOM-20120814-29270009.nii.gz


  2%|▏         | 26/1405 [30:10<26:58:55, 70.44s/it]2025-04-30 13:21:45.118 | DEBUG    | io3d.datareader:__ReadFromDirectory:261 - Getting list of readable files in dir...


Reading 500 files in /home/mjirik/mnt/nas-bmc3_ct/Prasata Tx/10_2019/Tx017A 4.10.2019/DICOM/20120814/29270010 and writing /home/mjirik/mnt/nas-bmc3_ct/pilsen_pigs_all/transplantation_nii_transposed/Prasata Tx-10_2019-Tx017A 4.10.2019-DICOM-20120814-29270010/Prasata Tx-10_2019-Tx017A 4.10.2019-DICOM-20120814-29270010.nii.gz


  2%|▏         | 26/1405 [30:59<27:23:53, 71.53s/it]


KeyboardInterrupt: 

In [ ]:
# import tqdm
# import time
# for i in tqdm.tqdm(range(100)):
#     #wait 0.1s
#     time.sleep(0.1)
    

In [ ]:
metafn = base_path / "meta.csv"

if metafn.exists():
    df = pd.read_csv(metafn)
else:
    df = pd.DataFrame()


In [ ]:
def create_meta(base_path, metafn):
    data = {
        "dirname": [],
        "dataset_type": [],
        "id": [],
    }
    i_train = 0
    i_test = 0
    i_val = 0
    fnlist = base_path.glob("*Tx0*D_V*")
    logger.info(pformat(list(map(str, list(fnlist)))))
    fnlist = base_path.glob("*Tx0*D_V*")
    for fn in fnlist:
        ia = int(re.findall(r"([0-9]+)", fn.name)[0])
        ii = None
        tp = None
        if (ia % 4) == 0:
            i_test += 1
            ii = i_test
            tp = "test"
        elif (ia % 2) == 0:
            i_val += 1
            ii = i_val
            tp = "val"
        else:
            i_train += 1
            ii = i_train
            tp = "train"

        data["dirname"].append(fn.name)
        data["dataset_type"].append(tp)
        data["id"].append(ii)

    df = pd.DataFrame(data)
    df.to_csv(metafn, index=None)



In [ ]:
if recreate_meta:
    create_meta(base_path, metafn)


In [ ]:

logger.debug(metafn.exists())
df = pd.read_csv(metafn)
print(df)


In [ ]:
# fnlist = base_path.glob("*debug*")
# fnlist = base_path.glob("*Tx041D_V*")


# for i, row in tqdm.tqdm(df.iterrows()):
#     fn_in = base_path / row["dirname"]
#     fn_out = (
#             output_path
#             / row["dataset_type"]
#             / f"PP_{row['id']:04}"
#             / "PATIENT_DICOM"
#             / f"PP_{row['id']:04}.mhd"
#     )
#     fn_meta = fn_out.parent / "meta.json"
#     logger.debug(fn_in)
#     logger.debug(fn_out)
#
#     fn_out.parent.mkdir(parents=True, exist_ok=True)
#     if (
#             not fn_meta.exists()
#     ):  # we do not need to read the data again if everything is done. We are checkin:w
#
#         tqdm.tqdm.write(f"Reading {fn_in} and writing {fn_out}")
#         datap = io3d.read(fn_in)
#         io3d.write(datap, fn_out)
#         with open(fn_meta, "w") as f:
#             json.dump(dict(row), f)
